## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [4]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "Genesis Scottish Open"
tournament_date = "7/13/2025"  # Ending date of tournament
old_course = "The Renaissance Club"
tournament_id = "R2025541"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "The Open Championship"
new_ending_date = "7/20/2025"
new_course = "Royal Portrush Golf Club"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [5]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025541 (Genesis Scottish Open), year: 20250
✅ 156 new rows added for 'Genesis Scottish Open'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2025,2025-07-13,R2025541,Genesis Scottish Open,The Renaissance Club,Chris Gotterup,1,1,-2,-9,E,-4,"$1,575,000.00",500.000
1,2025,2025-07-13,R2025541,Genesis Scottish Open,The Renaissance Club,Marco Penge,T2,2,-5,-3,-1,-4,"$788,175.00",0.000
2,2025,2025-07-13,R2025541,Genesis Scottish Open,The Renaissance Club,Rory McIlroy,T2,2,-2,-5,-4,-2,"$788,175.00",245.000
3,2025,2025-07-13,R2025541,Genesis Scottish Open,The Renaissance Club,Nicolai Hjgaard,T4,4,-2,-1,-3,-6,"$407,250.00",122.500
4,2025,2025-07-13,R2025541,Genesis Scottish Open,The Renaissance Club,Matt Fitzpatrick,T4,4,-1,-7,-1,-3,"$407,250.00",122.500


#### Stats

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,631,.1868,2025
1,Aaron Baddeley,172.0,-1.231,176.0,-1.271,159.0,-0.454,7.0,0.494,153.0,...,290.1,115.0,57.46%,175.0,59.69%,69.0,60.42%,497,.2656,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,336,.4307,2025
3,Aaron Rai,28.0,0.724,22.0,0.390,50.0,0.247,68.0,0.087,113.0,...,287.8,2.0,73.62%,25.0,69.26%,102.0,59.04%,36,2.6092,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,862,.1092,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

✅ Cleaned and updated 207 rows in 'odds' table.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG
5797,2024,The Open Championship,2024-07-21,Rory McIlroy,17/2,8.5,Open Championship,Rory McIlroy
5798,2024,The Open Championship,2024-07-21,Jon Rahm,20/1,20.0,Open Championship,Jon Rahm
5799,2024,The Open Championship,2024-07-21,Scottie Scheffler,11/2,5.5,Open Championship,Scottie Scheffler
5800,2024,The Open Championship,2024-07-21,Viktor Hovland,30/1,30.0,Open Championship,Viktor Hovland
5801,2024,The Open Championship,2024-07-21,Cameron Smith,50/1,50.0,Open Championship,Cameron Smith


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

✅ Inserting 120 new rows into odds table...


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [16]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [17]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [18]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 8 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
624,2016,Royal Troon GC,100,The Open Championship,2016-07-17
468,2017,Royal Birkdale GC,100,The Open Championship,2017-07-23
312,2018,Carnoustie GC,100,The Open Championship,2018-07-22
156,2019,Royal Portrush Golf Club,100,The Open Championship,2019-07-21
0,2021,Royal St. George's GC,100,The Open Championship,2021-07-18
935,2022,St. Andrews Links (Old Course),100,The Open Championship,2022-07-17
1091,2023,Royal Liverpool,R2023100,The Open Championship,2023-07-23
1247,2024,Royal Troon,R2024100,The Open Championship,2024-07-21


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [21]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-07-17 — The Open Championship (472 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,17,10,378.0,58.8,22.24,0,2016-07-17,The Open Championship
1,Aaron Wise,2,0,0.0,0.0,0.00,0,2016-07-17,The Open Championship
2,Abraham Ancer,10,2,43.0,20.0,4.30,2,2016-07-17,The Open Championship



📆 2017-07-23 — The Open Championship (456 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,6,224.79,42.9,16.06,0,2017-07-23,The Open Championship
1,Aaron Rai,1,0,0.00,0.0,0.00,0,2017-07-23,The Open Championship
2,Aaron Wise,5,4,0.00,80.0,0.00,4,2017-07-23,The Open Championship



📆 2018-07-22 — The Open Championship (465 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-07-17,The Open Championship
1,Aaron Baddeley,5,2,45.94,40.0,9.19,1,2022-07-17,The Open Championship
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-07-17,The Open Championship



📆 2023-07-23 — The Open Championship (581 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,17,11,349.536,64.7,20.56,0,2023-07-23,The Open Championship
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-07-23,The Open Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-07-23,The Open Championship



📆 2024-07-21 — The Open Championship (559 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,10,184.725,71.4,13.19,1,2024-07-21,The Open Championship
1,Aaron Cockerill,2,0,0.000,0.0,0.00,0,2024-07-21,The Open Championship
2,Aaron Rai,18,15,881.757,83.3,48.99,10,2024-07-21,The Open Championship


### Recent Form

In [22]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-07-17 — The Open Championship (472 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Lee McCoy,1,4.0,5.77,2016-07-17,The Open Championship
1,Cody Gribble,1,8.0,11.54,2016-07-17,The Open Championship
2,Dustin Johnson,11,11.4,4.59,2016-07-17,The Open Championship



📆 2017-07-23 — The Open Championship (456 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Keith Mitchell,1,11.0,15.87,2017-07-23,The Open Championship
1,Oscar Fraustro,1,13.0,18.76,2017-07-23,The Open Championship
2,Cam Davis,1,15.0,21.64,2017-07-23,The Open Championship



📆 2018-07-22 — The Open Championship (465 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Chris Gotterup,1,4.0,5.77,2022-07-17,The Open Championship
1,Haotong Li,1,12.0,17.31,2022-07-17,The Open Championship
2,Rory McIlroy,13,15.8,5.99,2022-07-17,The Open Championship



📆 2023-07-23 — The Open Championship (581 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Brooks Koepka,3,6.7,4.83,2023-07-23,The Open Championship
1,Scottie Scheffler,19,7.3,2.44,2023-07-23,The Open Championship
2,Grant Forrest,1,11.0,15.87,2023-07-23,The Open Championship



📆 2024-07-21 — The Open Championship (559 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,3,3.0,2.16,2024-07-21,The Open Championship
1,Romain Langasque,1,3.0,4.33,2024-07-21,The Open Championship
2,Scottie Scheffler,16,6.3,2.22,2024-07-21,The Open Championship


### Course History

In [23]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))

In [24]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

ℹ️ Empty course_hist for date 2019-07-21


## Training Dataset

In [25]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-07-17,The Open Championship,Royal Troon GC,Henrik Stenson,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,5.0,8.52,25.0,57.1,567.07,81.01,0.0,44.3,21.30,1
1,2016,2016-07-17,The Open Championship,Royal Troon GC,Phil Mickelson,2,2,26.0,0.799,134.0,-0.145,6.0,0.726,36.0,0.218,9.0,0.565,8.0,22.99,1.0,2.93,68.0,4.04,17.0,4.59,141.0,219.0,56.0,294.8,163.0,54.82,84.0,66.05,4.0,64.32,15.0,4.89,30.0,64.3,871.15,62.22,1.0,41.1,15.18,1
2,2016,2016-07-17,The Open Championship,Royal Troon GC,J.B. Holmes,3,3,38.0,0.633,7.0,0.663,138.0,-0.151,31.0,0.233,122.0,-0.129,10.0,22.82,35.0,3.03,68.0,4.04,93.0,4.68,86.0,180.0,1.0,314.5,179.0,51.68,105.0,65.17,130.0,56.68,22.0,4.01,150.0,83.3,695.27,57.94,1.0,28.3,11.03,1
3,2016,2016-07-17,The Open Championship,Royal Troon GC,Steve Stricker,4,4,94.0,0.101,164.0,-0.440,116.0,-0.003,1.0,0.544,5.0,0.620,80.0,20.23,35.0,3.03,14.0,4.01,42.0,4.63,149.0,228.0,175.0,278.0,53.0,63.23,158.0,63.0,1.0,66.01,82.0,1.79,150.0,50.0,170.60,21.32,1.0,61.1,27.81,1
4,2016,2016-07-17,The Open Championship,Royal Troon GC,Sergio Garcia,T5,5,9.0,1.189,4.0,0.737,24.0,0.482,112.0,-0.031,168.0,-0.388,16.0,21.9,16.0,3.01,48.0,4.03,28.0,4.61,9.0,98.0,28.0,299.9,70.0,61.76,6.0,70.15,113.0,57.66,13.0,5.04,20.0,85.7,967.92,138.27,6.0,36.9,17.75,1
5,2016,2016-07-17,The Open Championship,Royal Troon GC,Tyrrell Hatton,T5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,33.0,3.32,150.0,NaN,NaN,NaN,NaN,NaN,NaN,1
6,2016,2016-07-17,The Open Championship,Royal Troon GC,Rory McIlroy,T5,5,2.0,1.888,1.0,1.230,29.0,0.447,37.0,0.211,139.0,-0.207,2.0,25.27,136.0,3.09,8.0,4.00,1.0,4.46,6.0,86.0,9.0,306.8,77.0,61.06,11.0,69.62,88.0,58.7,3.0,11.07,10.0,77.8,488.35,54.26,0.0,31.4,13.64,1
7,2016,2016-07-17,The Open Championship,Royal Troon GC,Andrew Johnston,8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,79.0,1.83,300.0,100.0,0.00,0.00,2.0,48.0,43.69,1
8,2016,2016-07-17,The Open Championship,Royal Troon GC,Bill Haas,T9,9,34.0,0.693,81.0,0.068,54.0,0.272,14.0,0.353,130.0,-0.174,172.0,17.89,22.0,3.02,68.0,4.04,100.0,4.69,55.0,159.0,97.0,289.6,62.0,62.34,57.0,67.14,12.0,63.3,44.0,2.87,150.0,75.0,781.50,48.84,2.0,41.7,14.72,1
9,2016,2016-07-17,The Open Championship,Royal Troon GC,Dustin Johnson,T9,9,3.0,1.646,2.0,1.117,31.0,0.430,75.0,0.066,37.0,0.281,1.0,25.81,16.0,3.01,1.0,3.97,4.0,4.54,33.0,140.0,2.0,313.6,138.0,57.17,43.0,67.82,70.0,59.58,2.0,11.31,8.0,100.0,2030.40,184.58,11.0,11.4,4.59,1


In [ ]:
# Play with different years to see if I notice anything wrong (check Odds for tournament name mismatches)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
1091,2024,2024-07-21,The Open Championship,Royal Troon,Xander Schauffele,1,1,2.0,1.521,10.0,0.568,7.0,0.700,25.0,0.252,10.0,0.546,6.0,25.04%,1.0,2.93,2.0,3.92,15.0,4.50,12.0,112.0,38.0,308.6,74.0,61.57%,30.0,69.63%,1.0,70.49%,2.0,10.8280,12.0,100.0,3298.583,194.03,17.0,13.1,4.53,1
1092,2024,2024-07-21,The Open Championship,Royal Troon,Justin Rose,T2,2,151.0,-0.467,116.0,-0.065,152.0,-0.373,112.0,-0.029,91.0,0.022,150.0,20.67%,126.0,3.07,131.0,4.04,101.0,4.59,104.0,181.0,127.0,297.5,54.0,63.20%,161.0,63.30%,38.0,62.53%,39.0,2.4538,200.0,64.7,506.025,29.77,0.0,57.5,19.89,1
1093,2024,2024-07-21,The Open Championship,Royal Troon,Billy Horschel,T2,2,67.0,0.289,69.0,0.146,105.0,0.007,61.0,0.136,25.0,0.442,93.0,22.22%,4.0,2.97,107.0,4.02,15.0,4.50,60.0,157.0,108.0,299.4,49.0,63.76%,100.0,66.45%,27.0,63.62%,17.0,3.5729,250.0,64.7,636.917,37.47,0.0,48.4,16.75,1
1094,2024,2024-07-21,The Open Championship,Royal Troon,Thriston Lawrence,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,49.0,2.1696,600.0,0.0,0.000,0.00,0.0,90.0,81.92,1
1095,2024,2024-07-21,The Open Championship,Royal Troon,Russell Henley,5,5,26.0,0.646,103.0,0.014,34.0,0.343,20.0,0.289,40.0,0.304,114.0,21.69%,21.0,3.00,26.0,3.98,129.0,4.63,70.0,167.0,160.0,291.3,7.0,69.49%,136.0,64.99%,13.0,65.00%,15.0,3.7122,125.0,93.8,1370.575,85.66,8.0,28.4,10.02,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [27]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-07-17,The Open Championship,Royal Troon GC,Henrik Stenson,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,5.0,8.52,25.0,57.1,567.07,81.01,0.0,44.3,21.30,1
1,2016,2016-07-17,The Open Championship,Royal Troon GC,Phil Mickelson,2,2,26.0,0.799,134.0,-0.145,6.0,0.726,36.0,0.218,9.0,0.565,8.0,22.99,1.0,2.93,68.0,4.04,17.0,4.59,141.0,219.0,56.0,294.8,163.0,54.82,84.0,66.05,4.0,64.32,15.0,4.89,30.0,64.3,871.15,62.22,1.0,41.1,15.18,1
2,2016,2016-07-17,The Open Championship,Royal Troon GC,J.B. Holmes,3,3,38.0,0.633,7.0,0.663,138.0,-0.151,31.0,0.233,122.0,-0.129,10.0,22.82,35.0,3.03,68.0,4.04,93.0,4.68,86.0,180.0,1.0,314.5,179.0,51.68,105.0,65.17,130.0,56.68,22.0,4.01,150.0,83.3,695.27,57.94,1.0,28.3,11.03,1
3,2016,2016-07-17,The Open Championship,Royal Troon GC,Steve Stricker,4,4,94.0,0.101,164.0,-0.440,116.0,-0.003,1.0,0.544,5.0,0.620,80.0,20.23,35.0,3.03,14.0,4.01,42.0,4.63,149.0,228.0,175.0,278.0,53.0,63.23,158.0,63.0,1.0,66.01,82.0,1.79,150.0,50.0,170.60,21.32,1.0,61.1,27.81,1
4,2016,2016-07-17,The Open Championship,Royal Troon GC,Sergio Garcia,T5,5,9.0,1.189,4.0,0.737,24.0,0.482,112.0,-0.031,168.0,-0.388,16.0,21.9,16.0,3.01,48.0,4.03,28.0,4.61,9.0,98.0,28.0,299.9,70.0,61.76,6.0,70.15,113.0,57.66,13.0,5.04,20.0,85.7,967.92,138.27,6.0,36.9,17.75,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [28]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-07-17,The Open Championship,Royal Troon GC,Henrik Stenson,1,1,59.415808,0.535055,72.905498,0.182797,68.309278,0.24706,75.235395,0.103979,81.902062,0.110789,75.383162,21.760498,78.733677,3.047904,67.121993,4.016237,72.687285,4.61012,76.132302,172.689003,79.252577,299.243471,93.436426,61.015533,90.931271,66.475825,82.245704,59.887388,5.0,8.52,25.0,57.1,567.07,81.01,0.0,44.3,21.30,1
1,2016,2016-07-17,The Open Championship,Royal Troon GC,Phil Mickelson,2,2,26.000000,0.799000,134.000000,-0.145000,6.000000,0.72600,36.000000,0.218000,9.000000,0.565000,8.000000,22.990000,1.000000,2.930000,68.000000,4.040000,17.000000,4.59000,141.000000,219.000000,56.000000,294.800000,163.000000,54.820000,84.000000,66.050000,4.000000,64.320000,15.0,4.89,30.0,64.3,871.15,62.22,1.0,41.1,15.18,1
2,2016,2016-07-17,The Open Championship,Royal Troon GC,J.B. Holmes,3,3,38.000000,0.633000,7.000000,0.663000,138.000000,-0.15100,31.000000,0.233000,122.000000,-0.129000,10.000000,22.820000,35.000000,3.030000,68.000000,4.040000,93.000000,4.68000,86.000000,180.000000,1.000000,314.500000,179.000000,51.680000,105.000000,65.170000,130.000000,56.680000,22.0,4.01,150.0,83.3,695.27,57.94,1.0,28.3,11.03,1
3,2016,2016-07-17,The Open Championship,Royal Troon GC,Steve Stricker,4,4,94.000000,0.101000,164.000000,-0.440000,116.000000,-0.00300,1.000000,0.544000,5.000000,0.620000,80.000000,20.230000,35.000000,3.030000,14.000000,4.010000,42.000000,4.63000,149.000000,228.000000,175.000000,278.000000,53.000000,63.230000,158.000000,63.000000,1.000000,66.010000,82.0,1.79,150.0,50.0,170.60,21.32,1.0,61.1,27.81,1
4,2016,2016-07-17,The Open Championship,Royal Troon GC,Sergio Garcia,T5,5,9.000000,1.189000,4.000000,0.737000,24.000000,0.48200,112.000000,-0.031000,168.000000,-0.388000,16.000000,21.900000,16.000000,3.010000,48.000000,4.030000,28.000000,4.61000,9.000000,98.000000,28.000000,299.900000,70.000000,61.760000,6.000000,70.150000,113.000000,57.660000,13.0,5.04,20.0,85.7,967.92,138.27,6.0,36.9,17.75,1


In [29]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [30]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,14200
1,Rory McIlroy,11500
2,Jon Rahm,10400
3,Bryson DeChambeau,10100
4,Xander Schauffele,9700
5,Tommy Fleetwood,9500
6,Shane Lowry,9300
7,Ludvig Aberg,9100
8,Robert Macintyre,9000
9,Tyrrell Hatton,8900


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [32]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,The Open Championship,Scottie Scheffler,11/2,5.5
2,2025,The Open Championship,Rory McIlroy,15/2,7.5
3,2025,The Open Championship,Xander Schauffele,25/1,25.0
4,2025,The Open Championship,Jon Rahm,11/1,11.0
5,2025,The Open Championship,Ludvig Aberg,25/1,25.0
6,2025,The Open Championship,Collin Morikawa,35/1,35.0
7,2025,The Open Championship,Bryson DeChambeau,20/1,20.0
8,2025,The Open Championship,Viktor Hovland,25/1,25.0
9,2025,The Open Championship,Tommy Fleetwood,20/1,20.0
10,2025,The Open Championship,Brooks Koepka,65/1,65.0


## Cut Percentage and FedEx Points

In [33]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-07-20,The Open Championship
1,Aaron Baddeley,14,6,49.088,42.9,3.51,0,2025-07-20,The Open Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2025-07-20,The Open Championship
3,Aaron Rai,18,14,669.980,77.8,37.22,3,2025-07-20,The Open Championship
4,Aaron Wise,3,1,6.333,33.3,2.11,0,2025-07-20,The Open Championship
5,Adam Hadwin,20,13,214.432,65.0,10.72,0,2025-07-20,The Open Championship
6,Adam Long,1,0,0.000,0.0,0.00,0,2025-07-20,The Open Championship
7,Adam Schenk,20,9,270.904,45.0,13.55,0,2025-07-20,The Open Championship
8,Adam Scott,14,12,439.203,85.7,31.37,7,2025-07-20,The Open Championship
9,Adam Svensson,19,10,106.593,52.6,5.61,1,2025-07-20,The Open Championship


## Recent Form

In [34]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Carlos Ortiz,1,4.0,5.77,2025-07-20,The Open Championship
1,Scottie Scheffler,16,6.9,2.44,2025-07-20,The Open Championship
2,Jon Rahm,3,9.7,7.00,2025-07-20,The Open Championship
3,Kristoffer Reitan,1,13.0,18.76,2025-07-20,The Open Championship
4,Bubba Watson,1,14.0,20.20,2025-07-20,The Open Championship
5,Marco Penge,2,15.0,13.65,2025-07-20,The Open Championship
6,Taisei Shimizu,1,16.0,23.08,2025-07-20,The Open Championship
7,Ian Gilligan,1,16.0,23.08,2025-07-20,The Open Championship
8,Stewart Cink,1,17.0,24.53,2025-07-20,The Open Championship
9,Andy Sullivan,1,17.0,24.53,2025-07-20,The Open Championship


## Course History

In [35]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Wise,1,41.0,59.15,2025-07-20,Royal Portrush Golf Club,The Open Championship
1,Abraham Ancer,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship
2,Adam Hadwin,1,57.0,82.23,2025-07-20,Royal Portrush Golf Club,The Open Championship
3,Adam Scott,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship
4,Adria Arnaus,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship
5,Adrian Otaegui,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship
6,Alex Noren,1,11.0,15.87,2025-07-20,Royal Portrush Golf Club,The Open Championship
7,Alexander Bjork,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship
8,Alexander Levy,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship
9,Andrea Pavan,1,90.0,129.84,2025-07-20,Royal Portrush Golf Club,The Open Championship


## Merged Dataframe

In [36]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Rai,28.0,0.724,22.0,0.390,50.0,0.247,68.0,0.087,113.0,-0.047,90.0,21.67%,177.0,3.18,10.0,3.98,48.0,4.57,76.0,171,169.0,287.8,2.0,73.62%,25.0,69.26%,102.0,59.04%,36.0,2.6092,2025.0,80.0,77.8,669.980,37.22,3.0,38.8,13.18,NaN,NaN,6900
1,Adam Scott,42.0,0.512,32.0,0.322,90.0,0.070,57.0,0.120,103.0,-0.016,93.0,21.58%,140.0,3.10,134.0,4.05,37.0,4.56,128.0,203,81.0,303.5,122.0,56.77%,123.0,64.96%,124.0,57.62%,40.0,2.4616,2025.0,70.0,85.7,439.203,31.37,7.0,38.5,14.22,90.0,129.84,7300
2,Adrien Saddier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0,1.0797,2025.0,400.0,0.0,0.000,0.00,0.0,90.0,129.84,NaN,NaN,5200
3,Akshay Bhatia,77.0,0.239,103.0,0.016,16.0,0.566,164.0,-0.343,22.0,0.362,14.0,24.07%,3.0,2.97,55.0,4.01,166.0,4.70,135.0,207,156.0,294.8,51.0,62.38%,54.0,67.22%,161.0,55.08%,38.0,2.5083,2025.0,150.0,77.8,840.990,46.72,2.0,39.9,13.55,NaN,NaN,6700
4,Aldrich Potgieter,74.0,0.258,2.0,0.686,117.0,-0.102,162.0,-0.326,72.0,0.132,29.0,23.36%,94.0,3.07,81.0,4.02,141.0,4.66,30.0,136,1.0,328.2,135.0,55.78%,101.0,65.40%,173.0,52.19%,53.0,2.0898,2025.0,200.0,35.7,941.000,67.21,0.0,62.9,23.23,NaN,NaN,6300


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [37]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [38]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [39]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [40]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Rai,28.000000,0.724000,22.000000,0.390000,50.000000,0.247000,68.000000,0.087000,113.000000,-0.047000,90.000000,21.670000,177.000000,3.180000,10.000000,3.980000,48.00000,4.570000,76.000000,171.000000,169.000000,287.800000,2.000000,73.620000,25.000000,69.260000,102.000000,59.040000,36.0,2.6092,2025.0,80.0,77.80000,669.980,37.220000,3.00000,38.8,13.180000,51.565217,74.391739,6900
1,Adam Scott,42.000000,0.512000,32.000000,0.322000,90.000000,0.070000,57.000000,0.120000,103.000000,-0.016000,93.000000,21.580000,140.000000,3.100000,134.000000,4.050000,37.00000,4.560000,128.000000,203.000000,81.000000,303.500000,122.000000,56.770000,123.000000,64.960000,124.000000,57.620000,40.0,2.4616,2025.0,70.0,85.70000,439.203,31.370000,7.00000,38.5,14.220000,90.000000,129.840000,7300
2,Adrien Saddier,64.123288,0.407904,75.506849,0.123658,73.876712,0.181575,70.767123,0.102726,76.068493,0.123795,71.191781,22.269589,86.712329,3.062192,67.493151,4.013973,82.30137,4.599863,90.493151,179.150685,94.041096,301.847945,85.109589,60.338356,86.575342,66.238904,91.794521,59.519178,139.0,1.0797,2025.0,400.0,0.00000,0.000,0.000000,0.00000,90.0,129.840000,51.565217,74.391739,5200
3,Akshay Bhatia,77.000000,0.239000,103.000000,0.016000,16.000000,0.566000,164.000000,-0.343000,22.000000,0.362000,14.000000,24.070000,3.000000,2.970000,55.000000,4.010000,166.00000,4.700000,135.000000,207.000000,156.000000,294.800000,51.000000,62.380000,54.000000,67.220000,161.000000,55.080000,38.0,2.5083,2025.0,150.0,77.80000,840.990,46.720000,2.00000,39.9,13.550000,51.565217,74.391739,6700
4,Aldrich Potgieter,74.000000,0.258000,2.000000,0.686000,117.000000,-0.102000,162.000000,-0.326000,72.000000,0.132000,29.000000,23.360000,94.000000,3.070000,81.000000,4.020000,141.00000,4.660000,30.000000,136.000000,1.000000,328.200000,135.000000,55.780000,101.000000,65.400000,173.000000,52.190000,53.0,2.0898,2025.0,200.0,35.70000,941.000,67.210000,0.00000,62.9,23.230000,51.565217,74.391739,6300
5,Andrew Novak,49.000000,0.412000,67.000000,0.166000,98.000000,0.047000,38.000000,0.199000,37.000000,0.269000,121.000000,21.060000,156.000000,3.120000,120.000000,4.040000,37.00000,4.560000,157.000000,229.000000,112.000000,299.500000,117.000000,57.330000,168.000000,61.440000,67.000000,60.650000,31.0,2.6756,2025.0,150.0,66.70000,1583.125,65.960000,5.00000,43.6,13.550000,51.565217,74.391739,6200
6,Angel Hidalgo,64.123288,0.407904,75.506849,0.123658,73.876712,0.181575,70.767123,0.102726,76.068493,0.123795,71.191781,22.269589,86.712329,3.062192,67.493151,4.013973,82.30137,4.599863,90.493151,179.150685,94.041096,301.847945,85.109589,60.338356,86.575342,66.238904,91.794521,59.519178,1000.0,17.9550,2025.0,1000.0,65.49115,0.000,38.612743,2.19469,90.0,34.037522,51.565217,74.391739,5400
7,Antoine Rozner,89.000000,0.137000,115.000000,-0.026000,28.000000,0.454000,159.000000,-0.291000,129.000000,-0.113000,39.000000,22.880000,1.000000,2.960000,134.000000,4.050000,76.00000,4.600000,74.000000,170.000000,40.000000,308.400000,130.000000,56.300000,23.000000,69.280000,162.000000,54.960000,148.0,0.9949,2025.0,500.0,90.90000,173.796,15.800000,10.00000,50.9,20.480000,51.565217,74.391739,5600
8,Ben Griffin,21.000000,0.844000,46.000000,0.265000,32.000000,0.411000,46.000000,0.168000,50.000000,0.200000,52.000000,22.510000,26.000000,3.020000,33.000000,4.000000,26.00000,4.540000,67.000000,167.000000,73.000000,304.500000,94.000000,59.520000,20.000000,69.590000,74.000000,60.190000,16.0,3.9902,2025.0,80.0,77.80000,2021.463,74.870000,0.00000,37.6,11.280000,51.565217,74.391739,7300
9,Bri

## Correlations
**Last 4 Years**

In [41]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [42]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [43]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 37 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.90350026 0.93611513 0.93117897 0.91547102 0.93918859]
Average ROC AUC score: 0.93


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [46]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Scottie Scheffler,14200,0.727074,6.9,51.565217,5.5,100.0,1.0,4055.967,1.000000,1.000000,1.000000,25.000000,22.000000,3.000000,26.000000,1.000000,1.00000,15.000000,69.000000,44.000000,13.000000,2.000000
1,Sam Burns,7400,0.581448,33.1,51.565217,70.0,85.0,22.0,1246.382,105.000000,77.000000,136.000000,83.000000,1.000000,15.000000,48.000000,21.000000,48.00000,35.000000,55.000000,89.000000,93.000000,112.000000
2,Hideki Matsuyama,7500,0.571233,37.2,90.000000,80.0,83.3,12.0,1198.836,18.000000,132.000000,22.000000,5.000000,73.000000,55.000000,17.000000,33.000000,117.00000,163.000000,116.000000,131.000000,88.000000,7.000000
3,Ben Griffin,7300,0.564745,37.6,51.565217,80.0,77.8,16.0,2021.463,21.000000,46.000000,32.000000,46.000000,50.000000,52.000000,26.000000,33.000000,26.00000,67.000000,73.000000,94.000000,20.000000,74.000000
4,Rory McIlroy,11500,0.534512,17.6,90.000000,7.5,91.7,2.0,3189.036,10.000000,5.000000,68.000000,31.000000,4.000000,10.000000,12.000000,10.000000,13.00000,76.000000,2.000000,169.000000,58.000000,17.000000
5,Keegan Bradley,6700,0.517336,25.9,90.000000,100.0,94.1,7.0,1714.967,4.000000,28.000000,35.000000,10.000000,112.000000,43.000000,66.000000,10.000000,117.00000,20.000000,74.000000,48.000000,76.000000,81.000000
6,Tommy Fleetwood,9500,0.516227,23.3,2.000000,20.0,93.3,13.0,1716.648,3.000000,46.000000,9.000000,17.000000,56.000000,64.000000,150.000000,21.000000,48.00000,82.000000,136.000000,38.000000,119.000000,29.000000
7,Jason Day,7200,0.447835,32.6,90.000000,90.0,85.7,30.0,1070.313,57.000000,97.000000,100.000000,20.000000,64.000000,58.000000,94.000000,21.000000,48.00000,162.000000,134.000000,107.000000,85.000000,37.000000
8,Bryson DeChambeau,10100,0.444699,32.3,90.000000,20.0,66.7,15.0,0.000,64.123288,75.506849,73.876712,70.767123,76.068493,71.191781,86.712329,67.493151,82.30137,90.493151,94.041096,85.109589,86.575342,91.794521
9,Wyndham Clark,6700,0.431597,45.7,51.565217,125.0,78.9,28.0,490.373,97.000000,66.000000,158.000000,18.000000,59.000000,26.000000,140.000000,81.000000,29.00000,68.000000,23.000000,145.000000,120.000000,130.000000


### Save to CSV

In [47]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
